# Introduction

This notebook introduces the syntax of formulating a model in Gurobi, specifically showing  a variety of different ways to formulate a Gurobi optimization model.  The initial examples are shown for clarity and for the purposes of showing concrete examples of what the Gurobi statements accomplish.  Later examples are more robust for the purposes of constructing large models and also faster than earlier formulations.  Most of the latter formulations will be sufficinetly fast for problems of teh scale encountered in real applications.

# Most Basic Approach: Hard-Coded Coefficients

This is the simplest and most understandable approach as a first example, although it is infeasible for larger data sets/models.  We nonetheless use this as a starting point for understanding what the more complex approaches do in principal.

In [9]:
import gurobipy as grb

# create Gurobi model
m = grb.Model('Sherwood')

# Specify how to optimize and time limit (seconds)
m.ModelSense = grb.GRB.MAXIMIZE  # can be specified when the objective function is defined
m.setParam('TimeLimit',7200)

# Create decision variables
qs = m.addVar(vtype=grb.GRB.CONTINUOUS,name='qs', lb=0.0)
qc = m.addVar(vtype=grb.GRB.CONTINUOUS,name='qc', lb=0.0, ub=32.0)

# Update model to include variables and parameters
m.update()

# Create constraints
m.addLConstr(4.0 * qs + 3.0 * qc, grb.GRB.LESS_EQUAL, 120.0,"AssmbDeptHours")
m.addLConstr(8.0 * qs + 2.0 * qc, grb.GRB.LESS_EQUAL, 160.0,"FinishDeptHours")
m.addLConstr(qc, grb.GRB.LESS_EQUAL, 32.0,"CustomSales")

# Set objective function
m.setObjective(20.0 * qs + 10.0 * qc, grb.GRB.MAXIMIZE) # redundant model sense specification

# Update the model to incorporate objective and constraints
m.update()

# Optimize the model
m.optimize()

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-1650 v3 @ 3.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 2 columns and 5 nonzeros
Model fingerprint: 0xb36f83a7
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [3e+01, 3e+01]
  RHS range        [3e+01, 2e+02]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0000000e+02   1.997900e+01   0.000000e+00      0s
       2    5.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.000000000e+02


# Alternate Constraint Formulation

Gurobi offers multiple ways to code different aspects of objective functions and constraints.  the frmulation below, although it still uses the <code>m.addLConstr()</code> for formulating constraints permits using < <=, >, >=, and == to indicate the constraint sense as opposed to the <code>GRB</code> namespace senese operators.

In [10]:
import gurobipy as grb

# create Gurobi model
m = grb.Model('Sherwood')

# Specify how to optimize and time limit (seconds)
m.ModelSense = grb.GRB.MAXIMIZE
m.setParam('TimeLimit',7200)

# Create decision variables
qs = m.addVar(vtype=grb.GRB.CONTINUOUS,name='qs', lb=0.0)
qc = m.addVar(vtype=grb.GRB.CONTINUOUS,name='qc', lb=0.0, ub=32.0)

# Update model to include variables and parameters
m.update()

# Create constraints: 
''' NOTE ALTERNATE SYNTAX '''
m.addLConstr(4.0 * qs + 3.0 * qc <= 120.0,"AssmbDeptHours")
m.addLConstr(8.0 * qs + 2.0 * qc <= 160.0,"FinishDeptHours")
m.addLConstr(qc <= 32.0,"CustomSales")

# Set objective function
m.setObjective(20.0 * qs + 10.0 * qc, grb.GRB.MAXIMIZE)

# Update the model to incorporate objective and constraints
m.update()

# Optimize the model
m.optimize()

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-1650 v3 @ 3.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 2 columns and 5 nonzeros
Model fingerprint: 0xb36f83a7
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [3e+01, 3e+01]
  RHS range        [3e+01, 2e+02]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0000000e+02   1.997900e+01   0.000000e+00      0s
       2    5.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.000000000e+02


# A More Concise & Flexible Formulation for Larger Data

In this formulation the data are contained in Python variables, although they could as easily be in external data files or in a MySQL database.

Constraints and the objective function are specified with generators (similar to list comprehension) such that the coefficients need not be hard coded and the formulation syntax will work for a model of any size.

The <code>grb.quicksum()</code>  is a relatively fast way to sum the products of coefficients and decision variables.

In [15]:
import gurobipy as grb

''' Data in Python Lists '''
cnstr_coeff = [[4.0,3.0],[8.0,2.0],[0,1.0]]
cnstr_names = ['AssmbDeptHours','FinishDeptHours','CustomSales']
cnstr_rhs = [120.0,160.0,32.0]
obj_coeff = [20.0,10.0]
var_name = ['qs', 'qc']

# Create Gurobi model
m = grb.Model('Sherwood')

# Specify how to optimize and time limit (seconds)
m.ModelSense = grb.GRB.MAXIMIZE
m.setParam('TimeLimit',7200)     

# Create decision variables in a Python list
dvars = []
for i in range(len(obj_coeff)):
    dvars.append(m.addVar(vtype=grb.GRB.CONTINUOUS,name=var_name[i], lb=0.0))

# Update model to include variables and parameters
m.update()

# Create constraints in a loop
for i in range(len(cnstr_coeff)):
    m.addLConstr(grb.quicksum((cnstr_coeff[i][j] * dvars[j] for j in range(len(dvars)))), grb.GRB.LESS_EQUAL, cnstr_rhs[i], cnstr_names[i])

# Create objective function
m.setObjective(grb.quicksum(obj_coeff[i] * dvars[i] for i in range(len(dvars))), grb.GRB.MAXIMIZE)

# Update model to include constraints and objective function
m.update()

# Optimize the model
m.optimize()

''' Print decision variable values and other information '''
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.x}, LB = {var.lb}, UB = {var.ub}')

''' Print sensitivity analysis information on constraints '''
for c in m.getConstrs():
    print(f'Constraint: {c.ConstrName}, RHS = {c.RHS}, slack = {c.slack}, Limits = ({c.SARHSLow}, {c.SARHSUp})')

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-1650 v3 @ 3.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 2 columns and 5 nonzeros
Model fingerprint: 0x05ef4501
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 2e+02]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0000000e+02   1.997900e+01   0.000000e+00      0s
       2    5.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.000000000e+02
Variable: qs, Optimal Value = 14.999999999999998, LB = 0.0, UB = inf
Variable: qc, Optimal Value = 20.00000

# A Formulation Using Gurobi LinExpr

Formulating objective functions and constraints using <code>LinExpr()</code> is proportedly faster than using <code>quicksum()</code>.  It is also, perhaps, more succinct because the generators or list comprehension because one level of <code>for</code> loop or list comprehension are no longer needed.

This formation also uses list comprehension.

[Link to Gurobi Model.LinExpr](https://www.gurobi.com/documentation/current/refman/py_lex.html)

In [17]:
import gurobipy as grb

A = [[4.0,3.0],[8.0,2.0],[0,1.0]]
cnstr_names = ['AssmbDeptHours','FinishDeptHours','CustomSales']
b = [120.0,160.0,32.0]
c = [20.0,10.0]
var_name = ['qs', 'qc']

# Create Gurobi model
m = grb.Model('Sherwood')

# Specify how to optimize and time limit (seconds)
m.ModelSense = grb.GRB.MAXIMIZE
m.setParam('TimeLimit',7200)


# Create decision variables using list comprehension
x = [m.addVar(vtype=grb.GRB.CONTINUOUS, name=var_name[i], lb=0.0) for i in range(len(var_name))]

# Update model to include variables
m.update()

# Create constraints
#for i in range(len(cnstr_rhs)):
#    m.addLConstr(grb.LinExpr(cnstr_coeff[i], x), grb.GRB.LESS_EQUAL, cnstr_rhs[i],cnstr_names[i])
cnstr = [m.addLConstr(grb.LinExpr(A[i], x), grb.GRB.LESS_EQUAL, b[i], cnstr_names[i]) for i in range(len(b))]

# Create objective function
m.setObjective(grb.LinExpr(c, x), grb.GRB.MAXIMIZE)

# Update model to include constraints and objective function
m.update()

# Optimize the model
m.optimize()

''' Print decision variable values and other information '''
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.x}, LB = {var.lb}, UB = {var.ub}')

''' Print sensitivity analysis information on constraints '''
for c in m.getConstrs():
    print(f'Constraint: {c.ConstrName}, RHS = {c.RHS}, slack = {c.slack}, Limits = ({c.SARHSLow}, {c.SARHSUp})')

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-1650 v3 @ 3.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 2 columns and 5 nonzeros
Model fingerprint: 0x05ef4501
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 2e+02]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0000000e+02   1.997900e+01   0.000000e+00      0s
       2    5.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.000000000e+02
Variable: qs, Optimal Value = 14.999999999999998, LB = 0.0, UB = inf
Variable: qc, Optimal Value = 20.00000

# A Formulation Using numpy Arrays

Using <code>numpy</code> arrays for coefficients might be a bit faster and permit objective functions and constraints to be specified using matrix-vector math.  Thus, multiple constraints can be specified in a single statement.

New statements:
- <code>Model.addMVar()</code>
- <code>Model.addMConstr()</code>
- <code>Model.addConstrs()</code>

In [65]:
import gurobipy as grb
import numpy as np

A = np.array([[4.0,3.0],[8.0,2.0],[0,1.0]])
#cnstr_names = ['AssmbDeptHours','FinishDeptHours','CustomSales']
b = np.array([120.0,160.0,32.0])
c = np.array([20.0,10.0])
var_name = ['qs', 'qc']

# Create Gurobi model
m = grb.Model('Sherwood')

# Specify how to optimize and time limit (seconds)
m.setParam('TimeLimit',7200)


# Create matrix of decision variables
x = m.addMVar(shape=(2), vtype=grb.GRB.CONTINUOUS, name='q') 
#x = [m.addVar(vtype=grb.GRB.CONTINUOUS, name=var_name[i], lb=0.0) for i in range(len(var_name))]

# Update model to include variables
m.update()

# Create constraints
m.addMConstr(A, x,  grb.GRB.LESS_EQUAL, b, name='cnstr')

# Create objective function
''' LinExpr() works only with variables in Python lists and tuples, but not Gurobi MVar format '''
m.setObjective(grb.LinExpr(c, x.tolist()), grb.GRB.MAXIMIZE)

# Update model to include constraints and objective function
m.update()

# Optimize the model
m.optimize()

''' Print decision variable values and other information '''
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.x}, (LB,UB) = ({var.lb}, {var.ub})')

''' Print sensitivity analysis information on constraints '''
for c in m.getConstrs():
    print(f'Constraint: {c.ConstrName}, RHS = {c.RHS}, slack = {c.slack}, (LB,UB) = ({c.SARHSLow}, {c.SARHSUp})')

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-1650 v3 @ 3.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 2 columns and 5 nonzeros
Model fingerprint: 0x05ef4501
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 2e+02]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0000000e+02   1.997900e+01   0.000000e+00      0s
       2    5.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.000000000e+02
Variable: q[0], Optimal Value = 14.999999999999998, (LB,UB) = (0.0, inf)
Variable: q[1], Optimal Value = 20

# Python's Matrix Multiplier Operator, <code>@</code>, and Reading Model Data from Files

This formulation incorporates these alternatives and advancements:
- Uses the <code>@</code> Python operator, which performs matrix multiplication.
- Generates <code>numpy</code> arrays from text files

New statements:
- <code>Model.addConstrs()</code>

In [68]:
import gurobipy as grb
import numpy as np

A = np.genfromtxt('A.txt')
#cnstr_names = ['AssmbDeptHours','FinishDeptHours','CustomSales']
b = np.genfromtxt('b.txt')
c = np.genfromtxt('c.txt')
#var_name = ['qs', 'qc']

# Create Gurobi model
m = grb.Model('Sherwood')

# Specify how to optimize and time limit (seconds)
m.setParam('TimeLimit',7200)


# Create matrix of decision variables
x = m.addMVar(shape=(2,1), vtype=grb.GRB.CONTINUOUS, name='x') 

# Update model to include variables
m.update()

# Create constraints
m.addConstrs((A @ x <= b for i in range(1)), name='cnstr') # generator iterator required despite not needing it
''' Alternative statement with m.addMConstr() 
      - Requires a 1D numpy array '''
#m.addMConstr(A,x.reshape((2,)), grb.GRB.LESS_EQUAL, b, name='cnstr')

# Create objective function
m.setObjective(c @ x, grb.GRB.MAXIMIZE)

# Update model to include constraints and objective function
m.update()

# Optimize the model
m.optimize()

''' Print decision variable values and other information '''
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.x}, (LB,UB) = ({var.lb}, {var.ub})')

''' Print sensitivity analysis information on constraints '''
for c in m.getConstrs():
    print(f'Constraint: {c.ConstrName}, RHS = {c.RHS}, slack = {c.slack}, (LB,UB) = ({c.SARHSLow}, {c.SARHSUp})')

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-1650 v3 @ 3.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 2 columns and 5 nonzeros
Model fingerprint: 0x05ef4501
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 2e+02]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0000000e+02   1.997900e+01   0.000000e+00      0s
       2    5.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.000000000e+02
Variable: x[0,0], Optimal Value = 14.999999999999998, (LB,UB) = (0.0, inf)
Variable: x[1,0], Optimal Value 